In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wandb

In [ ]:
import wandb

wandb.login()

In [ ]:
from google.colab import drive
import zipfile
import os

# 2. 압축 파일 경로 설정
zip_file_path = '/content/drive/MyDrive/스마트팩토리_배터리/data/K-Fold.zip'
extract_folder = '/content/data'

# 3. 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("압축 해제가 완료되었습니다.")


In [ ]:
pip install scikit-learn

In [ ]:
pip install pyyaml

In [ ]:
pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO
import wandb

# 폴드 수와 경로 설정
num_folds = 5
#base_fold_path = 'K-Fold/folds'  # 폴드 디렉토리 경로
data_config_template = '/content/data/K-Fold/fold_{}/data.yaml'  # 각 폴드에 맞는 data.yaml 경로

# 각 폴드에 대해 모델 학습
for fold in range(1, num_folds):
    # WandB 프로젝트 설정

    ###########################################
    ########## name  번호 바꿔주기!! ##########
    ###########################################
    wandb.init(project='K-Fold', name='train_02')
    # YOLOv8 모델 로드
    model = YOLO('yolov8n.pt')

    # 각 폴드에 맞는 데이터셋 설정 파일 경로
    data_config = data_config_template.format(fold)

    print(f"폴드 {fold}에 대해 모델 학습 시작...")

    # 모델 학습
    ### 파라미터들 바꿔주기
    model.train(data=data_config, epochs=200, imgsz=270, batch=8, workers=12)

    print(f"폴드 {fold}에 대한 모델 학습이 완료되었습니다.")
    wandb.finish()

print("모든 폴드에 대한 모델 학습이 완료되었습니다.")


In [ ]:
import os

# CPU 코어 수 확인
cpu_cores = os.cpu_count()
print(f"사용 가능한 CPU 코어 수: {cpu_cores}")


In [ ]:
import os
from ultralytics import YOLO

# 3. 모델 평가 및 테스트
# 학습된 모델의 정확한 경로를 확인
model_dir = 'runs/detect/train2/weights'
model_path = os.path.join(model_dir, 'best.pt')

# 모델 경로 확인
if os.path.exists(model_path):
    model = YOLO(model_path)
    # 모델 평가 (검증 데이터셋 사용)
    results = model.val(data=data_config)
    print(results)
else:
    print(f"모델 파일을 찾을 수 없습니다: {model_path}")

# 테스트 이미지 디렉토리 설정
test_dir = '/content/data/YOLO_dataset/test/images'

# PNG와 JPG 파일을 모두 가져오기
images = [f for f in os.listdir(test_dir) if f.endswith('.png') or f.endswith('.jpg')]
for image in images:
    image_path = os.path.join(test_dir, image)
    results = model(image_path)

    # 예측 결과를 개별적으로 저장
    for idx, result in enumerate(results):
        save_path = f'/content/drive/MyDrive/smartFactory/Text/{os.path.splitext(image)[0]}_prediction_{idx}.png'
        result.save(filename=save_path)

print("예측 결과 저장이 완료되었습니다.")


In [ ]:
import shutil

# run 폴더를 Google 드라이브로 복사
source = '/content/runs'
#########################################
###########저장 경로 바꾸기 #############
#########################################
destination = '/content/drive/MyDrive/smartFactory/trian02'

# destination 디렉토리가 없으면 생성
os.makedirs(destination, exist_ok=True)

# 디렉토리 복사
shutil.copytree(source, destination, dirs_exist_ok=True)